In [21]:
from urllib.parse import quote
import os
import re
import time

import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [36]:
def extract_date_info(title):
    title = " ".join(title.split())
    pattern = r"tháng (\d+) năm (\d+)"
    match = re.search(pattern, title)
    
    if match:
        month = int(match.group(1))
        year = int(match.group(2))
        return month, year
    else:
        return None, None

def download_docx(title, document_link):

    response = requests.get(document_link)
    title = title.replace('/', ' năm ').replace('-', ' ')

    type = document_link.split('.')[-1]

    month, year = extract_date_info(title)
    if month:
        print(f'Month: {month}, Year: {year}')
        directory = f'../assets/doc/{year}/{month}'
        file_path = os.path.join(directory, f'{title}.{type}')
    else:
        directory = f'../assets/doc/error'
        file_path = os.path.join(directory, f'{title}.{type}')
    
    if not os.path.exists(directory):
        os.makedirs(directory)

    try:
        with open (file_path, 'wb') as f:
            f.write(response.content)
        return 1
    except:
        return 0

def get_document_link(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    title = soup.find('span', class_='title-article').text
    n_desc = soup.find('div', class_='n-desc')
    try:
        path = quote(n_desc.find('a').attrs['href'])
        document_link = 'https://www.ppd.gov.vn/' + path
        return title, document_link
    except:
        return title, None

In [37]:
driver = webdriver.Chrome()

# Mở trang web
driver.get('https://www.ppd.gov.vn/thong-bao-tinh-hinh-svgh-7-ngay.html')

wait = WebDriverWait(driver, 10)
pagination = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "pager")))

total_pages = int(driver.find_element(By.CLASS_NAME, 'last').text)

for page_number in range(1, total_pages + 1):
    
    print(f"Page {page_number}")

    page_source = driver.page_source

    soup = BeautifulSoup(page_source, 'html.parser')

    titles = soup.findAll('a', class_='title')

    urls = ['https://www.ppd.gov.vn' + link.attrs['href'] for link in titles]

    for url in urls:
        
        title, document_link = get_document_link(url)
        if document_link:
            result = download_docx(title, document_link)
            if result == 1:
                print(title)
            else:
                print(f'Error at Page {page_number} | Title: {title}')
        else:
            print(f'Error at Page {page_number} | Title: {title}')

    if page_number < total_pages:
        next_page_button = driver.find_element(By.XPATH, f"//a[@href=\"javascript:__doPostBack('ctl00$cphMain$ctl00$pager','{page_number}')\"]")
        next_page_button.click()

        time.sleep(2)

driver.quit()

Page 1
Month: 8, Year: 2024
Thông báo tình hình dịch hại tuần 01 tháng 08 năm 2024
Month: 7, Year: 2024
Thông báo tình hình dịch hại tuần 04 tháng 07 năm 2024
Month: 7, Year: 2024
Thông báo tình hình dịch hại tuần 03 tháng 07 năm 2024
Month: 7, Year: 2024
Thông báo tình hình dịch hại tuần 02 tháng 07 năm 2024
Month: 7, Year: 2024
Thông báo tình hình dịch hại tuần 01 tháng 07 năm 2024
Month: 6, Year: 2023
Thông báo tình hình dịch hại tuần 04 tháng 06 năm 2023
Month: 6, Year: 2024
Thông báo tình hình dịch hại tuần 03 tháng 06 năm 2024
Month: 6, Year: 2024
Thông báo tình hình dịch hại tuần 02 tháng 06 năm 2024
Month: 6, Year: 2024
Thông báo tình hình dịch hại tuần 01 tháng 06 năm 2024
Month: 5, Year: 2024
Thông báo tình hình dịch hại tuần 04 tháng 05 năm 2024
Page 2
Month: 5, Year: 2024
Thông báo tình hình dịch hại tuần 03 tháng 05 năm 2024
Month: 5, Year: 2024
Thông báo tình hình dịch hại tuần 02 tháng 5 năm 2024
Month: 4, Year: 2024
Thông báo tình hình dịch hại tuần 03 tháng 04 năm 2024